In [1]:
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from collections import defaultdict
from sklearn.decomposition import TruncatedSVD, NMF, SparsePCA
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def recall5(answer_df, submission_df):
    """
    Calculate recall@5 for given dataframes.
    
    Parameters:
    - answer_df: DataFrame containing the ground truth
    - submission_df: DataFrame containing the predictions
    
    Returns:
    - recall: Recall@5 value
    """
    
    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]
    
    # Check if each primary_col entry has exactly 5 secondary_col predictions
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # Check for NULL values in the predicted secondary_col
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")
    
    # Check for duplicates in the predicted secondary_col for each primary_col
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]
    
    # For each primary_col, get the top 5 predicted secondary_col values
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()
    
    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()
    
    
    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함 
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

In [3]:
apply_train = pd.read_csv('./apply_train.csv')

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def adjusted_cosine_similarity(matrix):
    # 각 행 또는 열의 평균을 구합니다.
    row_means = np.nanmean(matrix, axis=1, keepdims=True)
    col_means = np.nanmean(matrix, axis=0, keepdims=True)

    # NaN 값을 0으로 대체합니다.
    filled_matrix = np.where(np.isnan(matrix), 0, matrix)

    # 행과 열의 평균을 뺍니다.
    matrix_centered = filled_matrix - row_means - col_means + np.nanmean(matrix)

    # 코사인 유사도 계산
    similarity = cosine_similarity(matrix_centered)
    return similarity


In [5]:
#학습, 검증 분리
train, val = [], []
apply_train_groupby = apply_train.groupby('resume_seq')['recruitment_seq'].apply(list)
for uid, iids in zip(apply_train_groupby.index.tolist(), apply_train_groupby.values.tolist()):
    for iid in iids[:-1]:
        train.append([uid,iid])
    val.append([uid, iids[-1]])

In [6]:
train = pd.DataFrame(train, columns=['resume_seq', 'recruitment_seq'])
val = pd.DataFrame(val, columns=['resume_seq', 'recruitment_seq'])
pred = apply_train.copy()

In [7]:
train_user_item_matrix = train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
pred_user_item_matrix = pred.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)

In [8]:
# # 훈련 데이터에 대한 조정 코사인 유사도 계산
# train_user_similarity = adjusted_cosine_similarity(train_user_item_matrix)
# train_item_similarity = adjusted_cosine_similarity(train_user_item_matrix.T)

# # 예측 데이터에 대한 조정 코사인 유사도 계산
# pred_user_similarity = adjusted_cosine_similarity(pred_user_item_matrix)
# pred_item_similarity = adjusted_cosine_similarity(pred_user_item_matrix.T)

In [9]:
train_user_similarity = cosine_similarity(train_user_item_matrix)
train_item_similarity = cosine_similarity(train_user_item_matrix.T)

pred_user_similarity = cosine_similarity(pred_user_item_matrix)
pred_item_similarity = cosine_similarity(pred_user_item_matrix.T)

In [10]:
train_user_predicted_scores = train_user_similarity.dot(train_user_item_matrix)
train_item_predicted_scores = train_user_item_matrix.dot(train_item_similarity)

pred_user_predicted_scores = pred_user_similarity.dot(pred_user_item_matrix)
pred_item_predicted_scores = pred_user_item_matrix.dot(pred_item_similarity)

In [11]:
# alpha = 0.98
# train_recommendations = []
# for idx, user in tqdm(enumerate(train_user_item_matrix.index)):
#     applied_jobs = set(train_user_item_matrix.loc[user][train_user_item_matrix.loc[user] == 1].index)
    
#     # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
#     sorted_job_indices = (train_item_predicted_scores.loc[user].values * alpha + train_user_predicted_scores[idx]).argsort()[::-1]
#     recommended_jobs = [job for job in train_user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
#     for job in recommended_jobs:
#         train_recommendations.append([user, job])

In [13]:
import numpy as np

alpha_values = np.arange(0.30, 1.0, 0.01)  

best_alpha = None
best_recall = 0

for alpha in alpha_values:
    train_recommendations = []
    for idx, user in tqdm(enumerate(train_user_item_matrix.index)):
        applied_jobs = set(train_user_item_matrix.loc[user][train_user_item_matrix.loc[user] == 1].index)
        
        # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
        sorted_job_indices = (train_item_predicted_scores.loc[user].values * alpha + train_user_predicted_scores[idx]).argsort()[::-1]
        recommended_jobs = [job for job in train_user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
        
        for job in recommended_jobs:
            train_recommendations.append([user, job])
    
    val_prediction = pd.DataFrame(train_recommendations, columns=['resume_seq', 'recruitment_seq'])
    recall = recall5(val, val_prediction)
    
    print(f"Alpha: {alpha}, Recall@5: {recall}")

    if recall > best_recall:
        best_recall = recall
        best_alpha = alpha

print(f"Best Alpha: {best_alpha}, Best Recall@5: {best_recall}")


0it [00:00, ?it/s]

8482it [00:12, 683.91it/s]


Alpha: 0.3, Recall@5: 0.1279179438811601


8482it [00:12, 669.91it/s]


Alpha: 0.31, Recall@5: 0.1280358406036312


...

Alpha: 0.9700000000000006, Recall@5: 0.12909691110587126


8482it [00:10, 783.72it/s]


Alpha: 0.9800000000000006, Recall@5: 0.12909691110587126


8482it [00:11, 763.09it/s]


Alpha: 0.9900000000000007, Recall@5: 0.12909691110587126
Best Alpha: 0.9700000000000006, Best Recall@5: 0.12909691110587126


In [73]:
val_prediction = pd.DataFrame(train_recommendations, columns=['resume_seq', 'recruitment_seq'])

In [74]:
recall5(val,val_prediction)

0.1279179438811601

In [75]:
recall5(val,val_prediction)

0.1279179438811601

In [54]:
alpha = 0.98
pred_recommendations = []
for idx, user in tqdm(enumerate(pred_user_item_matrix.index)):
    applied_jobs = set(pred_user_item_matrix.loc[user][pred_user_item_matrix.loc[user] == 1].index)
    
    sorted_job_indices = (pred_item_predicted_scores.loc[user].values * alpha + pred_user_predicted_scores[idx]).argsort()[::-1]
    recommended_jobs = [job for job in pred_user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        pred_recommendations.append([user, job])

0it [00:00, ?it/s]

8482it [00:23, 367.43it/s]


In [55]:
top_recommendations = pd.DataFrame(pred_recommendations, columns=['resume_seq', 'recruitment_seq'])
top_recommendations.to_csv('E:/국민대/open/models/baseline_add_item_0.97.csv', index=False)